In [1]:
import sys
sys.path.append('..')
from lib.model_utils import ModelUtil
utils = ModelUtil(data_name = 'german', model_name = 'random forest')

Test
Accuracy Score is 0.828
Precision Score is 0.8663101604278075
F1 Score is 0.88283378746594
Train
Accuracy Score is 0.8932692307692308


In [2]:
mat = utils.get_cover_matrix(utils.X, fuzzy = True)

In [6]:
mat.sum(axis = 1).mean()

119.89066098333959